In [8]:
import torch
print(torch.cuda.is_available())          # Should return True
print(torch.cuda.get_device_name(0))      # Should show "RTX 3050"
print(torch.version.cuda)                 # Should match CUDA 12.1 (compatible with 12.8 driver)

True
NVIDIA GeForce RTX 3050 6GB Laptop GPU
12.1


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image

# Check for CUDA (GPU support)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class ChestXRayDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform
        self.labels = [
            'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion',
            'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass',
            'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax'
        ]

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        
        # Extract labels (0 or 1 for each pathology)
        labels = self.dataframe.iloc[idx][self.labels].values.astype(np.float32)
        
        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(labels)
    
class ChestXRayModel(nn.Module):
    def __init__(self, num_classes=15):
        super(ChestXRayModel, self).__init__()
        
        self.densenet = models.densenet121(pretrained=True)
        
        # Replace the classifier
        num_features = self.densenet.classifier.in_features
        self.densenet.classifier = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        x = self.densenet(x)
        return x  # BCEWithLogitsLoss applies sigmoid internally
    
# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load CSV and split into train/val/test
df = pd.read_csv("Data_Entry.csv")
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

# Create datasets
image_dir = "images/"
train_dataset = ChestXRayDataset(train_df, image_dir, transform=transform)
val_dataset = ChestXRayDataset(val_df, image_dir, transform=transform)
test_dataset = ChestXRayDataset(test_df, image_dir, transform=transform)

# Create dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        all_labels = []
        all_preds = []
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                
                # Store for metrics
                all_labels.append(labels.cpu().numpy())
                all_preds.append(torch.sigmoid(outputs).cpu().numpy())
        
        # Calculate epoch metrics
        train_loss = train_loss / len(train_loader.dataset)
        val_loss = val_loss / len(val_loader.dataset)
        
        # Concatenate predictions and labels
        all_labels = np.concatenate(all_labels)
        all_preds = np.concatenate(all_preds)
        
        # Compute ROC-AUC
        roc_auc = roc_auc_score(all_labels, all_preds, average='macro')
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val ROC-AUC: {roc_auc:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_model.pth")
        
        # Adjust learning rate
        scheduler.step(val_loss)
    
    print("Training complete!")

def evaluate_model(model, test_loader, labels):
    model.eval()
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            
            all_labels.append(labels.cpu().numpy())
            all_preds.append(torch.sigmoid(outputs).cpu().numpy())
    
    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)
    
    # Compute ROC-AUC for each class
    roc_auc_scores = []
    for i in range(all_labels.shape[1]):
        roc_auc = roc_auc_score(all_labels[:, i], all_preds[:, i])
        roc_auc_scores.append(roc_auc)
        print(f"Class {i} ({model.labels[i]}): ROC-AUC = {roc_auc:.4f}")
    
    print(f"Average ROC-AUC: {np.mean(roc_auc_scores):.4f}")

if __name__ == "__main__":
    # Initialize model
    model = ChestXRayModel(num_classes=15).to(device)
    
    # Loss function (BCEWithLogitsLoss for multi-label)
    criterion = nn.BCEWithLogitsLoss()
    
    # Optimizer & Scheduler
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)
    
    # Train the model
    train_model(model, criterion, optimizer, scheduler, num_epochs=10)
    
    # Evaluate on test set
    print("\nEvaluating on test set...")
    model.load_state_dict(torch.load("best_model.pth"))
    evaluate_model(model, test_loader)

Using device: cuda


c:\Users\prasa\OneDrive\Desktop\asthma project\myenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\prasa\OneDrive\Desktop\asthma project\myenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\prasa\OneDrive\Desktop\asthma project\myenv\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/10
Train Loss: 0.1921, Val Loss: 0.1800, Val ROC-AUC: 0.7078
Epoch 2/10
Train Loss: 0.1848, Val Loss: 0.1760, Val ROC-AUC: 0.7404
Epoch 3/10
Train Loss: 0.1813, Val Loss: 0.1754, Val ROC-AUC: 0.7349
Epoch 4/10
Train Loss: 0.1786, Val Loss: 0.1796, Val ROC-AUC: 0.7421
Epoch 5/10
Train Loss: 0.1765, Val Loss: 0.1741, Val ROC-AUC: 0.7591
Epoch 6/10
Train Loss: 0.1747, Val Loss: 0.1710, Val ROC-AUC: 0.7647
Epoch 7/10
Train Loss: 0.1724, Val Loss: 0.1694, Val ROC-AUC: 0.7706
Epoch 8/10
Train Loss: 0.1708, Val Loss: 0.1682, Val ROC-AUC: 0.7830
Epoch 9/10
Train Loss: 0.1687, Val Loss: 0.1677, Val ROC-AUC: 0.7853
Epoch 10/10
Train Loss: 0.1665, Val Loss: 0.1686, Val ROC-AUC: 0.7849
Training complete!

Evaluating on test set...


C:\Users\prasa\AppData\Local\Temp\ipykernel_1704\386484965.py:188: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


AttributeError: 'ChestXRayModel' object has no attribute 'labels'

In [8]:
model = ChestXRayModel().to(device)
model.load_state_dict(torch.load("best_model.pth"))
print("Model loaded successfully!")

c:\Users\prasa\OneDrive\Desktop\asthma project\myenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\prasa\OneDrive\Desktop\asthma project\myenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\prasa\AppData\Local\Temp\ipykernel_1704\1044566419.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpick

Model loaded successfully!


In [11]:
labels = [
            'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion',
            'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass',
            'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax'
        ]

def evaluate_model(model, test_loader, labels):
    model.eval()
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            
            all_labels.append(labels.cpu().numpy())
            all_preds.append(torch.sigmoid(outputs).cpu().numpy())
    
    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)
    
    # Compute ROC-AUC for each class
    roc_auc_scores = []
    for i in range(all_labels.shape[1]):
        roc_auc = roc_auc_score(all_labels[:, i], all_preds[:, i])
        roc_auc_scores.append(roc_auc)
        print(f"Class {i} ({labels[i]}): ROC-AUC = {roc_auc:.4f}")
    
    print(f"Average ROC-AUC: {np.mean(roc_auc_scores):.4f}")

evaluate_model(model,test_loader, labels)

Class 0 (tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       device='cuda:0')): ROC-AUC = 0.7967
Class 1 (tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       device='cuda:0')): ROC-AUC = 0.9211
Class 2 (tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       device='cuda:0')): ROC-AUC = 0.7674
Class 3 (tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       device='cuda:0')): ROC-AUC = 0.7800
Class 4 (tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       device='cuda:0')): ROC-AUC = 0.8940
Class 5 (tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       device='cuda:0')): ROC-AUC = 0.8485
Class 6 (tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')): ROC-AUC = 0.7017
Class 7 (tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       device='cuda:0')): ROC-AUC = 0.7394
Class 8 (tensor([0., 0., 0., 0.,

In [12]:
def evaluate_model(model, test_loader):
    model.eval()
    all_labels = []
    all_preds = []
    
    # Define class names (must match training order!)
    class_names = [
        'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion',
        'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass',
        'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax'
    ]
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            
            all_labels.append(labels.cpu().numpy())
            all_preds.append(torch.sigmoid(outputs).cpu().numpy())
    
    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)
    
    # Compute ROC-AUC for each class
    roc_auc_scores = []
    for i in range(len(class_names)):
        roc_auc = roc_auc_score(all_labels[:, i], all_preds[:, i])
        roc_auc_scores.append(roc_auc)
        print(f"{class_names[i]}: ROC-AUC = {roc_auc:.4f}")  # Now shows names
    
    print(f"\nAverage ROC-AUC: {np.mean(roc_auc_scores):.4f}")

evaluate_model(model,test_loader)

Atelectasis: ROC-AUC = 0.7967
Cardiomegaly: ROC-AUC = 0.9211
Consolidation: ROC-AUC = 0.7674
Edema: ROC-AUC = 0.7800
Effusion: ROC-AUC = 0.8940
Emphysema: ROC-AUC = 0.8485
Fibrosis: ROC-AUC = 0.7017
Hernia: ROC-AUC = 0.7394
Infiltration: ROC-AUC = 0.6361
Mass: ROC-AUC = 0.8344
No Finding: ROC-AUC = 0.7591
Nodule: ROC-AUC = 0.7185
Pleural_Thickening: ROC-AUC = 0.7495
Pneumonia: ROC-AUC = 0.6694
Pneumothorax: ROC-AUC = 0.8590

Average ROC-AUC: 0.7783
